<a href="https://colab.research.google.com/github/dev23m/brur-contest/blob/main/codelandAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name =  "Salesforce/codet5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [3]:
input_text = "return "  # Replace with your actual input
inputs = tokenizer(input_text, return_tensors="pt")


In [4]:
# Generate predictions
outputs = model.generate(
    inputs['input_ids'],
    max_length=50,  # Adjust this value as needed
    num_beams=5,    # For beam search; you can use num_beams=1 for greedy decoding
    early_stopping=True
)
predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(predicted_text)


function ( e ) {


In [7]:

from datasets import load_dataset
import json

# Load your JSON data, specifying the correct path
with open('/content/drive/MyDrive/codelanderAI datasets/dataset1.json', 'r') as f:
    data = json.load(f)

# Convert JSON to Dataset format
dataset = load_dataset('json', data_files={'train': '/content/drive/MyDrive/codelanderAI datasets/dataset1.json'}) # Removed extra '.json' from the path

In [8]:
def tokenize_function(examples):
    inputs = tokenizer(examples['input'], truncation=True, padding='max_length', max_length=128)
    targets = tokenizer(examples['target'], truncation=True, padding='max_length', max_length=128)
    inputs['labels'] = targets['input_ids']
    return inputs


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments

# Set up the training arguments with optimized settings
training_args = TrainingArguments(
    output_dir='./results',
    # Remove evaluation or add eval dataset
    # evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Increase batch size if GPU memory allows
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,  # Enable mixed precision training
    gradient_accumulation_steps=4,  # Adjust based on batch size
    dataloader_num_workers=4,  # Increase for faster data loading
)

# Initialize the Trainer with the adjusted training arguments
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    # Add evaluation dataset if you want to evaluate the model
    # eval_dataset=tokenized_datasets['validation'] # Assuming you have a 'validation' split in your dataset
)

# Start the fine-tuning process
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Step,Training Loss


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
# Save the trained model and tokenizer
model.save_pretrained('./codelander')
tokenizer.save_pretrained('./codelander')
